# Asking Questions about External Websites using Google Bard

This notebook contains Python code that uses [Google Bard](https://bard.google.com/chat) to inquire about external websites. It imports data from a public [Google Sheet](https://docs.google.com/spreadsheets/d/1_4VN5MQVPO6KK14mH0P8zBz25s7a-he67qS6Fst6ZTo/edit?usp=sharing), extracts a column containing links to genomics tutorials and tools, then requests Google Bard to give brief descriptions of these external links. To do all this in Python, I use an unofficial python package, [bardapi](https://github.com/dsdanielpark/Bard-API), which returns responses from Google Bard through cookies.

First, we import the [gspread](https://github.com/burnash/gspread) Python package to work with Google Sheets. Then, We set up authentication with Google Sheets and retrieve the Google Sheet.

In [1]:
import gspread

gc = gspread.oauth()
sh = gc.open_by_url("https://docs.google.com/spreadsheets/d/1_4VN5MQVPO6KK14mH0P8zBz25s7a-he67qS6Fst6ZTo/edit?usp=sharing")
worksheet = sh.get_worksheet(0)

/Users/howardbaek/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Then, we extract the column containing links to genomics tutorials and links. We print out the first 3 values to check our code.

In [9]:
tutorials_and_tool_links = worksheet.col_values(10)
tutorials_and_tool_links.remove("tutorials_and_tool_links")
print(tutorials_and_tool_links[0:3])

['https://informatics.fas.harvard.edu/atac-seq-guidelines.html', 'https://informatics.fas.harvard.edu/atac-seq-guidelines.html', 'https://training.galaxyproject.org/training-material/topics/epigenetics/tutorials/atac-seq/slides.html#1']


To interact with the Google Bard API, we import the bardapi package and set up authentication with cookie values. See this [issue](https://github.com/dsdanielpark/Bard-API/issues/155#issuecomment-1681499652) for more information. Currently, this method of using cookies is unstable, and fails after a few attempts. This is not the fault of the Python package; Google is making this API difficult to access.

In [10]:
from bardapi import BardCookies

description = []

cookie_dict = {
    "__Secure-1PSID": "YOUR_COOKIE_VALUE",
    "__Secure-1PSIDTS": "YOUR_COOKIE_VALUE",
    "__Secure-1PSIDCC": "YOUR_COOKIE_VALUE"
}
bard = BardCookies(cookie_dict = cookie_dict)

for link in tutorials_and_tool_links:
    print("Start!-----")
    print("Working on link: ", link)
    answer = bard.get_answer(f"Give me a short description of this website: {link}")['content']
    print("Description of link: ", answer)
    description.append(answer)
    print("Done!-----")


Exception: SNlM0e value not found in response. Check __Secure-1PSID value.

In the future, if this cookie method of authentication works and we are able to get as many responses from the Google Bard API, we can proceed to the following step. The following code fetches a range of cells, updates their content with descriptions from a list, and then applies those changes back to the Google Sheets worksheet.

In [ ]:
cell_list = worksheet.range("K" + '2:' + "K" + str(1 + len(tutorials_and_tool_links)))

for i, cell in enumerate(cell_list):
    cell.value = description[i]

# Update the worksheet with the modified data
worksheet.update_cells(cell_list)

{'spreadsheetId': '1_4VN5MQVPO6KK14mH0P8zBz25s7a-he67qS6Fst6ZTo',
 'updatedRange': 'main!K2:K77',
 'updatedRows': 76,
 'updatedColumns': 1,
 'updatedCells': 76}